# Clusterização de Endereços

Este notebook agrupa variações de um mesmo endereço sob um único UID (ID Único de Endereço), mesmo quando há diferenças na grafia.

In [ ]:
# Importar configurações
%run ./00_configuracao_inicial.ipynb

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer, NGram
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
import hashlib

# Importar funções de similaridade (será criado em notebook separado)
# %run ./05_algoritmos_similaridade.ipynb

In [ ]:
# UDF para calcular similaridade Jaro-Winkler (simplificado)
def jaro_winkler_similarity(s1, s2):
    """Calcula similaridade Jaro-Winkler entre duas strings"""
    if not s1 or not s2:
        return 0.0
    
    s1 = s1.upper().strip()
    s2 = s2.upper().strip()
    
    if s1 == s2:
        return 1.0
    
    # Implementação simplificada do Jaro-Winkler
    # Para produção, usar biblioteca como jellyfish ou similar
    len_s1 = len(s1)
    len_s2 = len(s2)
    
    match_window = max(len_s1, len_s2) // 2 - 1
    if match_window < 0:
        match_window = 0
    
    s1_matches = [False] * len_s1
    s2_matches = [False] * len_s2
    
    matches = 0
    transpositions = 0
    
    # Encontrar matches
    for i in range(len_s1):
        start = max(0, i - match_window)
        end = min(i + match_window + 1, len_s2)
        
        for j in range(start, end):
            if s2_matches[j] or s1[i] != s2[j]:
                continue
            s1_matches[i] = True
            s2_matches[j] = True
            matches += 1
            break
    
    if matches == 0:
        return 0.0
    
    # Contar transposições
    k = 0
    for i in range(len_s1):
        if not s1_matches[i]:
            continue
        while not s2_matches[k]:
            k += 1
        if s1[i] != s2[k]:
            transpositions += 1
        k += 1
    
    jaro = (matches / len_s1 + matches / len_s2 + (matches - transpositions / 2) / matches) / 3.0
    
    # Winkler prefix bonus
    prefix = 0
    for i in range(min(len(s1), len(s2), 4)):
        if s1[i] == s2[i]:
            prefix += 1
        else:
            break
    
    winkler = jaro + (0.1 * prefix * (1 - jaro))
    
    return min(1.0, winkler)

udf_jaro_winkler = udf(jaro_winkler_similarity, DoubleType())

In [ ]:
# Função para agrupar endereços similares por UF+Cidade
def clusterizar_enderecos(df_camada_ouro, threshold_similaridade=0.85):
    """
    Agrupa endereços similares em clusters baseado em similaridade de string
    
    Args:
        df_camada_ouro: DataFrame com Camada Ouro
        threshold_similaridade: Threshold mínimo de similaridade para agrupar (0-1)
    
    Returns:
        DataFrame com clusters de endereços
    """
    # Criar chave de comparação (UF + Cidade + Tipo + Nome Logradouro)
    df_com_chave = df_camada_ouro \
        .withColumn("chave_comparacao", 
            concat_ws("|", 
                col("uf"),
                col("cidade"),
                col("tipo_logradouro"),
                col("nome_logradouro")
            )
        )
    
    # Para cada registro, encontrar matches similares
    # Usar self-join com condição de similaridade
    df_renamed = df_com_chave.alias("a")
    df_compare = df_com_chave.alias("b")
    
    # Join apenas em mesma UF+Cidade para reduzir comparações
    df_similar = df_renamed.join(
        df_compare,
        (col("a.uf") == col("b.uf")) & 
        (col("a.cidade") == col("b.cidade")) &
        (col("a.uid") != col("b.uid")),
        "inner"
    ) \
    .withColumn("similaridade", 
        udf_jaro_winkler(col("a.nome_logradouro"), col("b.nome_logradouro"))
    ) \
    .filter(col("similaridade") >= threshold_similaridade) \
    .select(
        col("a.uid").alias("uid_origem"),
        col("b.uid").alias("uid_destino"),
        col("similaridade")
    )
    
    return df_similar

In [ ]:
# Função para criar clusters conectados (usando grafos)
def criar_clusters_conectados(df_similar):
    """
    Cria clusters de endereços conectados por similaridade
    Usa algoritmo de componentes conectados
    
    Args:
        df_similar: DataFrame com pares de UIDs similares
    
    Returns:
        DataFrame com cluster_id para cada UID
    """
    from graphframes import GraphFrame
    
    # Criar vértices (todos os UIDs únicos)
    vertices = df_similar.select("uid_origem").union(df_similar.select("uid_destino")) \
        .distinct() \
        .withColumnRenamed("uid_origem", "id")
    
    # Criar arestas (relações de similaridade)
    edges = df_similar.select(
        col("uid_origem").alias("src"),
        col("uid_destino").alias("dst"),
        col("similaridade").alias("weight")
    )
    
    # Criar GraphFrame
    graph = GraphFrame(vertices, edges)
    
    # Encontrar componentes conectados
    clusters = graph.connectedComponents()
    
    # Renomear coluna de cluster
    df_clusters = clusters.withColumnRenamed("component", "cluster_id")
    
    return df_clusters

# Alternativa sem GraphFrames (usando algoritmo simples)
def criar_clusters_simples(df_similar, df_camada_ouro):
    """
    Versão simplificada sem GraphFrames
    Agrupa por maior similaridade
    """
    from pyspark.sql.window import Window
    
    # Para cada UID, encontrar o melhor match
    window_spec = Window.partitionBy("uid_origem").orderBy(desc("similaridade"))
    
    df_best_match = df_similar \
        .withColumn("rank", row_number().over(window_spec)) \
        .filter(col("rank") == 1) \
        .select(
            col("uid_origem").alias("uid"),
            col("uid_destino").alias("cluster_representante")
        )
    
    # Criar cluster_id baseado no representante
    df_clusters = df_camada_ouro.join(
        df_best_match,
        "uid",
        "left"
    ) \
    .withColumn("cluster_id", 
        coalesce(col("cluster_representante"), col("uid"))  # Se não tem match, cluster = próprio UID
    ) \
    .select("uid", "cluster_id")
    
    return df_clusters

In [ ]:
# Função principal de clusterização
def executar_clusterizacao(threshold_similaridade=0.85):
    """
    Executa processo completo de clusterização
    
    Args:
        threshold_similaridade: Threshold de similaridade para agrupar
    
    Returns:
        DataFrame com clusters
    """
    print("1. Carregando Camada Ouro...")
    df_camada_ouro = read_delta_table(PATH_CAMADA_OURO)
    print(f"   Total de registros: {df_camada_ouro.count()}")
    
    print("2. Identificando endereços similares...")
    df_similar = clusterizar_enderecos(df_camada_ouro, threshold_similaridade)
    print(f"   Total de pares similares: {df_similar.count()}")
    
    print("3. Criando clusters...")
    df_clusters = criar_clusters_simples(df_similar, df_camada_ouro)
    print(f"   Total de clusters: {df_clusters.select('cluster_id').distinct().count()}")
    
    # Adicionar informações da Camada Ouro aos clusters
    df_clusters_completo = df_clusters.join(df_camada_ouro, "uid", "inner") \
        .withColumn("clusterizado_em", current_timestamp())
    
    return df_clusters_completo

In [ ]:
# Executar clusterização
# df_clusters = executar_clusterizacao(threshold_similaridade=0.85)

# Visualizar amostra
# df_clusters.show(20, truncate=False)

# Estatísticas de clusters
# df_clusters.groupBy("cluster_id").agg(
#     count("*").alias("tamanho_cluster")
# ).agg(
#     avg("tamanho_cluster").alias("tamanho_medio"),
#     max("tamanho_cluster").alias("tamanho_max")
# ).show()

In [ ]:
# Salvar clusters
# save_delta_table(df_clusters, PATH_CLUSTERS, mode="overwrite", partition_by=["uf", "cidade"])

print("Clusterização concluída!")